In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from transformers import GPT2Tokenizer, GPT2Model
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features with GPT-2 embeddings
def process_features_gpt2(input_file, text_column, numeric_features, model_name="gpt2"):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Load GPT-2 model and tokenizer
    print("Loading GPT-2 model and tokenizer...")
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Add the padding token to the tokenizer
    tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token

    model = GPT2Model.from_pretrained(model_name)

    # Generate GPT-2 embeddings for each text input
    print("Generating GPT-2 embeddings...")
    embeddings = []
    for text in df[text_column].astype(str):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding="max_length")
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the mean pooling of the last hidden state as the sentence embedding
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(embedding)
    embeddings = np.array(embeddings)
    print(f"GPT-2 embeddings shape: {embeddings.shape}")

    # Extract numeric features and combine with embeddings
    numeric_df = df[numeric_features]
    combined_features = np.hstack([embeddings, numeric_df.to_numpy()])

    return pd.DataFrame(combined_features), df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Define model functions
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'solver': ['liblinear', 'saga'], 'max_iter': [100, 200]}
    model = tune_model(LogisticRegression(), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'SVM': svm_model,
        'Logistic Regression': logistic_regression_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Process the features with GPT-2
X, original_data = process_features_gpt2(input_file, text_column, numeric_features)

# Prepare target variable with shifted labels
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models with hyperparameter tuning
run_all_models(X_train, y_train, X_test, y_test)


Mounted at /content/drive
Loading GPT-2 model and tokenizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Best Params: {'kernel': 'linear', 'gamma': 'scale', 'C': 1}
SVM Model Metrics:
Train Accuracy: 0.9013
Train Precision: 0.9014
Train Recall: 0.9013
Train F1 Score: 0.8977
Train Confusion Matrix:
[[ 22   5   5]
 [  2 101  29]
 [  1   5 306]]

Test Accuracy: 0.7415
Test Precision: 0.7385
Test Recall: 0.7415
Test F1 Score: 0.7400
Test Confusion Matrix:
[[  3   4   6]
 [  4  20  17]
 [  5  17 129]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=30. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'max_iter': 100, 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.9538
Train Precision: 0.9548
Train Recall: 0.9538
Train F1 Score: 0.9532
Train Confusion Matrix:
[[ 28   1   3]
 [  0 117  15]
 [  0   3 309]]

Test Accuracy: 0.7268
Test Precision: 0.7282
Test Recall: 0.7268
Test F1 Score: 0.7264
Test Confusion Matrix:
[[  1   5   7]
 [  5  22  14]
 [  3  22 126]]

Running Decision Tree model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 20}
Decision Tree Model Metrics:
Train Accuracy: 0.9181
Train Precision: 0.9215
Train Recall: 0.9181
Train F1 Score: 0.9192
Train Confusion Matrix:
[[ 27   4   1]
 [  4 119   9]
 [  5  16 291]]

Test Accuracy: 0.5756
Test Precision: 0.6265
Test Recall: 0.5756
Test F1 Score: 0.5968
Test Confusion Matrix:
[[  0   4   9]
 [  6  15  20]
 [  9  39 103]]

Running Naive Bayes model...
Naive Bayes Model Metrics:
Train Accuracy: 0.3718
Train Precision: 0.6137
Train Recall: 0.3718
Train F1 Score: 0.3989
Train Confusion Matrix:
[[ 22   7   3]
 [ 46  65  21]
 [ 86 136  90]]

Test Accuracy: 0.3415
Test Precision: 0.6426
Test Recall: 0.3415
Test F1 Score: 0.3923
Test Confusion Matrix:
[[ 7  4  2]
 [15 17  9]
 [44 61 46]]

